In [1]:
import pandas as pd
import re

In [2]:
data=pd.read_csv('../week3/Political-media-DFE.csv', encoding='latin1')

In [3]:
data.columns

Index(['_unit_id', '_golden', '_unit_state', '_trusted_judgments',
       '_last_judgment_at', 'audience', 'audience:confidence', 'bias',
       'bias:confidence', 'message', 'message:confidence', 'orig__golden',
       'audience_gold', 'bias_gold', 'bioid', 'embed', 'id', 'label',
       'message_gold', 'source', 'text'],
      dtype='object')

> * Subset the DataFrame to have `label`, `source`, and `text` columns

In [4]:
df=data[['label', 'source', 'text']]

In [5]:
df.shape

(5000, 3)

> * Q. How many unique users are there in the dataset? (check the `label` column)

In [6]:
df['label'].nunique()

505

> * Let's print the first 5 rows of the DataFrame

In [7]:
df.head(5)

,label,source,text
0,From: Trey Radel (Representative from Florida),twitter,RT @nowthisnews: Rep. Trey Radel (R- #FL) slam...
1,From: Mitch McConnell (Senator from Kentucky),twitter,VIDEO - #Obamacare: Full of Higher Costs and ...
2,From: Kurt Schrader (Representative from Oregon),twitter,Please join me today in remembering our fallen...
3,From: Michael Crapo (Senator from Idaho),twitter,RT @SenatorLeahy: 1st step toward Senate debat...
4,From: Mark Udall (Senator from Colorado),twitter,.@amazon delivery #drones show need to update ...


> * Let's extract the name of the user from the `label` column and create a new column called `username` in the DataFrame

In [8]:
pattern=re.compile(r'(.+?)\s*\(')
df['username']=df['label'].str.replace('From: ', '').str.findall(pattern).str[0]

/var/folders/3_/r8z51q092v566bd_r0g_zj640000gn/T/ipykernel_3619/1246208881.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['username']=df['label'].str.replace('From: ', '').str.findall(pattern).str[0]


>* Let's convert the usernames to lowercase

In [9]:
df['username']=df['username'].str.lower()

/var/folders/3_/r8z51q092v566bd_r0g_zj640000gn/T/ipykernel_3619/3231511176.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['username']=df['username'].str.lower()


In [10]:
df['username']

0            trey radel
1       mitch mcconnell
2         kurt schrader
3         michael crapo
4            mark udall
             ...       
4995           ted yoho
4996           ted yoho
4997           ted yoho
4998           ted yoho
4999           ted yoho
Name: username, Length: 5000, dtype: object

In [11]:
df.reset_index(drop=True, inplace=True)

> * Let's extract the mentions from the `text` column and create a new column `mention` with the mentions.

In [ ]:
pattern = re.compile()
df['mention']=df['text'].apply(lambda x: pattern.findall(x))
df['mention']=df['mention'].apply(lambda x: [y.lower() for y in x])

>* We also don't want to forget to convert the text into lowercase.

In [12]:
df['text']=df['text'].str.lower()

/var/folders/3_/r8z51q092v566bd_r0g_zj640000gn/T/ipykernel_3619/473300915.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text']=df['text'].str.lower()


In [ ]:
df.head(5)

> * Let's build edges between the users who have mentioned.
> * To do so, we will use the `mentions` column and iterate over the rows to create edges between the users in the `mention` column.

In [ ]:
#first we need to iterate through the mention column and then we need to iterate through usernames column
#and we need to create a tuple of the username and the mention

edges=[]
for idx, val in df.iterrows():
    if len(val['mention']) == 0: #when there are no mentions, we skip the iteration
        continue
    elif len(val['mention']) > 0: #when there are mentions, we iterate through the mentions
        for mention in val['mention']: 
            edges.append((val['username'].lower(), mention.strip('@').lower())) 
            #we append the tuple of the username and the mention to the edges list
            #lower() is used to convert the username to lowercase
            #strip() is used to remove the @ from the mention

In [ ]:
edges[:10]

In [ ]:
len(edges)

In [ ]:
edges[:10]

> * Let's count the degree centrality of each user and create a dictionary called `degree` for degree centrality.

>* First, you iterate through the unique users and count the number of edges that are connected to the user.

In [ ]:
degree={}
for element in df['username'].unique():
    count = 0
    for edge in edges:
        if element in edge:
            count += 1
    degree[element]=count


>* Let's check whether it captured the degree by looking at 'trey radel' instance.

In [ ]:
degree['trey radel']

In [ ]:
for edge in edges:
    if 'trey radel' in edge:
        print(edge)

> * We have five degree for 'trey radel'. Let's divide the degree by the number of unique users -1 to calculate degree centrality.

<img src="../week5/degree-centrality.png" width=1000px height=200px />

> * How do we want to find the number of nodes in the graph?

In [ ]:
#first, we try the number of unique usernames in df.T
degree['trey radel']/(len(df['username'].unique())-1)

> * But this is incorrect answer. Why?

In [ ]:
import networkx as nx
G=nx.Graph()
G.add_edges_from(edges)


In [ ]:
nx.degree_centrality(G)['trey radel']

> * As we talked before, the number of nodes in the graph will be larger than the number of unique users. Because we have to consider the users who have mentioned in the dataset but not in the `label` column.

>* So, we have to find the number of unique users in the edge list. 

In [ ]:
degree['trey radel']/(len(set([element for tuple in edges for element in tuple]))-1)

> * Now we know how to calculate the degree centrality. Let's iterate through the `degree` dictionary and divide the degree by the number of unique users in the edge list.

In [ ]:
for key, val in degree.items():
    degree[key]=val/(len(set([element for tuple in edges for element in tuple]))-1)

In [ ]:
degree['trey radel']

>* Q. What can be other attributes for the nodes?

>* There is an information about whether the political leader is a Representative or a Senator.
>* Let's use RegEx to extract that and create a dictionary called `title`.

In [ ]:
df['label']

In [ ]:
pattern=re.compile()
df['title']=df['label'].str.findall(pattern).str[0].str.strip('(')

In [ ]:
df['title'].value_counts()

In [ ]:
title={}
for idx, row in df.iterrows():
    title[row['username']]=row['title']

>* There is another information about which State the political leader is from.
>* Let's use RegEx to extract that and create a dictionary called `state`.

>* We are changing the value of Representative and Senator to 'o' and 'x' to use it as a node shape.

In [ ]:
title_shape = {key: 'o' if value == 'Representative' else 'x' for key, value in title.items()}

In [ ]:
pattern=re.compile(r'from\s.*')
df['state']=df['label'].str.findall(pattern).str[0].str.replace('from','').str.replace(')', '').str.strip()

In [ ]:
state={}
for idx, row in df.iterrows():
    state[row['username']]=row['state']


In [ ]:
df['state'].value_counts()[:10]

In [ ]:
import random
import seaborn as sns
# Get the unique values from the state dictionary
unique_states = list(set(state.values()))

# Generate a color palette using seaborn
color_palette = sns.color_palette("hls", len(unique_states))

# Create a dictionary to map each unique state to a color
state_colors_dict = {key: color_palette[i] for i, key in enumerate(unique_states)}


In [ ]:
color_palette

In [ ]:
state_colors_dict['Florida']

In [ ]:
df[df['username']=='trey radel']

In [ ]:
from collections import defaultdict
default_dict=defaultdict(lambda: None, state)
for key, value in state.items():
    if value in state_colors_dict.keys():
        default_dict[key]=state_colors_dict.get(value)
color_mapped=dict(default_dict)

In [ ]:
color_mapped['trey radel']

>* Practice

In [ ]:
data=pd.read_csv('subset-2021-01-11-voter_fraud.csv')

>* Let's see what it has for column names.

In [ ]:
#YOUR CODE HERE

>* Let's subset the data to have only `bodywithurls`, `username`, `followers`, and `following` columns.

In [ ]:
#YOUR CODE HERE

>* Q. How many unique users are there in the dataset?

In [ ]:
#YOUR CODE HERE

>* Q. Print the first 5 rows of the DataFrame.

>* Q. In the result of the first 5 rows of the DataFrame, do you see any duplicates?

In [ ]:
#YOUR CODE HERE

> * Looks like there are duplicates in the dataset. Let's remove the duplicates.

In [ ]:
#YOUR CODE HERE

> * Q. How many rows and columns are there in the dataset after removing the duplicates?

>* Once we remove the duplicates, let's reset the index of the DataFrame.

In [ ]:
#YOUR CODE HERE

> * Let's extract the mentions from the `bodywithurls` column and create a new column `mentions` with the mentions.

In [ ]:
#YOUR CODE HERE

>* Q. Which account has mentioned the most number of users?

In [ ]:
#YOUR CODE HERE

>* Q. Who are mentioned by that account?

In [ ]:
#YOUR CODE HERE

> * Let's build edges between the users who have mentioned.
> * To do so, we will use the `mentions` column and iterate over the rows to create edges between the users in the `mention` column.

In [ ]:
#YOUR CODE HERE

>* How many edges are there in the data?

In [ ]:
#YOUR CODE HERE